# Company Events Calendar

In [1]:
import ipywidgets as widgets
import pandas as pd
import datetime as dt
import os
import eikon as ek
# replace this with your Eikon API key
EIKON_APP_KEY = os.environ.get('EIKON_APP_KEY')
if EIKON_APP_KEY==None:
    print('App key is needed to retrieve data from Eikon. Generate your app key and use it in ek.set_app_key call.')
else:
    ek.set_app_key(EIKON_APP_KEY)

In [2]:
event_types_map = {'Brokerage analyst calls':'BACALL',
                   'Brokerage analyst meetings':'BAM',
                   'Company visits':'CV',
                   'Conference presentations':'CONFP',
                   'Conferences':'CONF',
                   'Corporate analyst meetings':'CAM',
                   'Corporate calls and presentations':'CCALL',
                   'Earnings calls and presentations':'ECALL',
                   'Earnings releases':'RES',
                   'Ex Dividends':'EXDIV',
                   'General & political events':'GPE',
                   'Guidance calls and presentations':'GCALL',
                   'IPO filings':'IPOF',
                   'IPO lockup expiration':'IPOLE',
                   'IPO pricings':'IPOP',
                   'IPO withdrawals':'IPOW',
                   'Market holidays':'MH',
                   'Merger & Acquisition calls and presentations':'MACALL',
                   'Other brokerages':'OTHB',
                   'Other Corporate events':'OTHC',
                   'Sales & Trading statement calls and presentations':'STCALL',
                   'Sales & Trading statement releases':'STR',
                   'Secondary filings':'SECF',
                   'Secondary pricings':'SECP',
                   'Secondary withdrawals':'SECW',
                   'Shareholder / Annual meetings':'SHM',
                   'Stock splits':'SSP',
                   'Syndicate roadshows':'SYNR'}
event_types_w = widgets.SelectMultiple(options = event_types_map, 
                                       rows=len(event_types_map),
                                       layout = widgets.Layout(width='50%'))
print('Select event types')
print('Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.')
print('No selection means retrieve all available event types')
display(event_types_w)

Select event types
Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.
No selection means retrieve all available event types


SelectMultiple(layout=Layout(width='50%'), options={'Brokerage analyst calls': 'BACALL', 'Brokerage analyst me…

In [3]:
event_types = event_types_w.value
if len(event_types) == 0:
    event_types_str = 'ALL'
else:
    event_types_str = event_types[0]
    for i in range(1,len(event_types)):
        event_types_str = event_types_str + ';' + event_types[i]
print('Select the date range for company events')
options_range = ['-10Y','-5Y','-3Y','-1Y','-6M','-3M','-1M','-1W','-1D','0D','1D','1W','2W','1M','3M','1Y']
date_range_w = widgets.SelectionRangeSlider(options=[' ' + x + ' ' for x in options_range],index=(9,13))
display(date_range_w)

Select the date range for company events


SelectionRangeSlider(index=(9, 13), options=(' -10Y ', ' -5Y ', ' -3Y ', ' -1Y ', ' -6M ', ' -3M ', ' -1M ', '…

In [4]:
start_date = options_range[date_range_w.index[0]]
end_date = options_range[date_range_w.index[1]]
instr, err = ek.get_data('0#.DJI','TR.IndexConstituentRIC')
instr = instr['Constituent RIC'].tolist() 
# Remove duplicates, None and empty strings from the list
instr = list(dict.fromkeys(instr))
instr = list(filter(None, instr))
instr = list(filter(str.strip, instr))

events_df, err = ek.get_data(instr,['TR.EventStartDate','TR.EventType','TR.EventTitle'],
                            {'SDate':start_date, 'EDate':end_date, 'EventType':event_types_str})
events_df.rename(columns = {'Event Start Date':'Event Date'}, inplace = True)
events_df = events_df[['Event Date','Instrument','Company Event Type','Event Title']]
events_df['Event Date'] = pd.to_datetime(events_df['Event Date'])
events_df.dropna(subset = ['Event Date'], inplace=True)
events_df

,Event Date,Instrument,Company Event Type,Event Title
1,2019-12-20,NKE.N,EarningsReleases,Q2 2020 Nike Inc Earnings Release
4,2019-12-13,DIS.N,ExDividends,Dividend For DIS.N - 0.8800 USD
5,2019-12-13,DIS.N,ExDividends,Dividend For DISB34.SA - 3.7074 BRL
6,2019-12-13,DIS.N,ExDividends,Dividend For DISN.BA - 0.2200 USD
9,2019-12-13,MRK.N,ExDividends,Dividend For MRK.N - 0.6100 USD
10,2019-12-13,MRK.N,ExDividends,Dividend For MRCK34.SA - 2.5671 BRL
11,2019-12-13,MRK.N,ExDividends,Dividend For MRK.BA - 0.1220 USD
